<a href="https://colab.research.google.com/github/gupta-keshav/age_gender_classification/blob/master/EfficientNet_alignedGenderSubmit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%pip install patool
import patoolib
patoolib.extract_archive("/content/drive/My Drive/Adience/alignedGender.rar", outdir="/content")

     |████████████████████████████████| 81kB 2.2MB/s 
patool: Extracting /content/drive/My Drive/Adience/alignedGender.rar ...
patool: running /usr/bin/unrar x -- "/content/drive/My Drive/Adience/alignedGender.rar"
patool:     with cwd='/content'
patool: ... /content/drive/My Drive/Adience/alignedGender.rar extracted to `/content'.


'/content'

In [ ]:
import keras
import  tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, PReLU, ReLU, Input, GlobalAveragePooling2D
from keras import optimizers, regularizers
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
np.random.seed(48)
from keras.applications import vgg19
from keras.models import Model, load_model
from keras.optimizers import Adam
# input_shape = (224, 224, 3)
# vgg = vgg19.VGG19(include_top=False ,weights='imagenet', 
#                                      input_shape=input_shape)   
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau

Using TensorFlow backend.


In [ ]:
!pip install efficientnet
import efficientnet.keras as efn 

In [ ]:
from efficientnet.keras import preprocess_input
class FixedImageDataGenerator(ImageDataGenerator):
  def standardize(self, x):
    if self.featurewise_center:
      img = preprocess_input(x)
      # x = ((x/255.) - 0.5) * 2.
    return x

In [ ]:
IMG_WIDTH = 256
IMG_HEIGHT = 256
CHANNELS = 3

In [ ]:
data_gen_args = dict(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
)

train_datagen = FixedImageDataGenerator(**data_gen_args)
train_generator = train_datagen.flow_from_directory(
        '/content/alignedGender/train',
        target_size=(IMG_WIDTH, IMG_HEIGHT),
        batch_size=32,
        class_mode='binary')

data_gen_args = dict(
    featurewise_center=True,
    featurewise_std_normalization=True,
    # rotation_range=20,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # horizontal_flip=True,
)
validate_datagen = FixedImageDataGenerator(**data_gen_args)
validate_generator = validate_datagen.flow_from_directory(
        '/content/alignedGender/test',
        target_size=(IMG_WIDTH, IMG_HEIGHT),
        batch_size=32,
        class_mode='binary')

Found 14047 images belonging to 2 classes.
Found 3445 images belonging to 2 classes.


In [ ]:
def efficientnetB0(model_input):
    baseModel = efn.EfficientNetB0(weights='imagenet', include_top=False, input_tensor=model_input)
    # for layer in baseModel.layers:
	#     layer.trainable = False
    # for i, layer in enumerate(baseModel.layers):
    #     if "batch_normalization" in layer.name:
    #         baseModel.layers[i] = GroupNormalization(groups=32, axis=-1, epsilon=0.00001)
    X = baseModel.output
    X = GlobalAveragePooling2D()(X)
    X = Dense(4096, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=0.1, l2=0.1))(X)
    X = Dropout(0.5)(X)
    X = Dense(2, activation="softmax")(X)
    model = Model(inputs=model_input, outputs=X)
    model.compile(optimizer=Adam(lr=1e-3, beta_1=0.9), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model_input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, CHANNELS))
efficientnetb0_model = efficientnetB0(model_input)

16809984/16804768 [==============================] - 0s 0us/step


In [ ]:
# efficientnetb0_model = load_model('/content/drive/My Drive/Adience/efficientnet.hdf5')
# efficientnetb0_model.load_weights('/content/drive/My Drive/Adience/effinet-adience-91.hdf5')

In [ ]:
filepath= "effinet-gender-v2.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_accuracy", mode="max", patience=5)
rlr = ReduceLROnPlateau(monitor='val_loss', 
                        factor=0.5, 
                        patience=2, 
                        verbose=1, 
                        mode='auto', 
                        min_delta=0.0001)
callbacks_list = [checkpoint, early, rlr]

In [ ]:
batch_size = 32
epochs = 10
history = efficientnetb0_model.fit_generator(train_generator, 
                              steps_per_epoch = train_generator.samples // batch_size, 
                              epochs = epochs,
                              validation_data = validate_generator,
                              validation_steps= validate_generator.samples // batch_size, 
                              callbacks = callbacks_list,
                              verbose=1)

Epoch 1/10
438/438 [==============================] - 441s 1s/step - loss: 345.9186 - accuracy: 0.8454 - val_loss: 65.3416 - val_accuracy: 0.7979

Epoch 00001: val_accuracy improved from -inf to 0.79790, saving model to effinet-gender-v2.hdf5
Epoch 2/10
438/438 [==============================] - 412s 940ms/step - loss: 63.5230 - accuracy: 0.8690 - val_loss: 62.7724 - val_accuracy: 0.8687

Epoch 00002: val_accuracy improved from 0.79790 to 0.86874, saving model to effinet-gender-v2.hdf5
Epoch 3/10
438/438 [==============================] - 410s 936ms/step - loss: 62.3212 - accuracy: 0.8818 - val_loss: 61.5368 - val_accuracy: 0.8579

Epoch 00003: val_accuracy did not improve from 0.86874
Epoch 4/10
438/438 [==============================] - 405s 925ms/step - loss: 61.7689 - accuracy: 0.8951 - val_loss: 60.1763 - val_accuracy: 0.8954

Epoch 00004: val_accuracy improved from 0.86874 to 0.89540, saving model to effinet-gender-v2.hdf5
Epoch 5/10
438/438 [==============================] - 398

In [ ]:
!cp  '/content/effinet-gender-v2-928.hdf5' '/content/drive/My Drive/Adience'

In [ ]:
last = vgg.output
print(vgg.output)

Tensor("block5_pool/MaxPool:0", shape=(?, 7, 7, 512), dtype=float32)


In [ ]:
x = Flatten()(last)
# x = keras.layers.BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(4096, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.1, l2=0.1))(x)
# x = keras.layers.BatchNormalization()(x)
x = Dropout(0.55)(x)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
preds = Dense(1, activation = 'sigmoid')(x)

In [ ]:
model = Model(vgg.input, preds)

In [ ]:
import pandas as pd
layers = [(layer.output, layer.name, layer.trainable) for layer in model.layers]
pd.set_option('max_colwidth', -1)

pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable']) 

In [ ]:
for i in range(17):
  model.layers[i].trainable = False

In [ ]:
efficientnetb0_model.compile(optimizer = optimizers.Adam(lr=1e-4), loss= 'binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
import tensorflow as tf
def preprocess(img):
  img = tf.keras.applications.vgg19.preprocess_input(img)
  return img

In [ ]:
data_gen_args = dict(
    featurewise_center=True,
    featurewise_std_normalization=True,
    # rotation_range=20,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # horizontal_flip=True,
)

train_datagen = FixedImageDataGenerator(**data_gen_args)
train_generator = train_datagen.flow_from_directory(
        '/content/alignedGender/train',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary')
validate_generator = train_datagen.flow_from_directory(
        '/content/alignedGender/test',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary')

Found 14047 images belonging to 2 classes.
Found 3445 images belonging to 2 classes.


In [ ]:
history = efficientnetb0_model.fit_generator(train_generator, steps_per_epoch=440, epochs=3, verbose=1, callbacks=None, validation_data=validate_generator, validation_steps=110, validation_freq=1, class_weight=None, max_queue_size=10, workers=-1, use_multiprocessing=True, shuffle=False)

In [ ]:
history = efficientnetb0_model.fit_generator(train_generator, steps_per_epoch=440, epochs=2, verbose=1, callbacks=None, validation_data=validate_generator, validation_steps=110, validation_freq=1, class_weight=None, max_queue_size=10, workers=-1, use_multiprocessing=True, shuffle=False)

In [ ]:
history = model.fit_generator(train_generator, steps_per_epoch=440, epochs=1, verbose=1, callbacks=None, validation_data=validate_generator, validation_steps=110, validation_freq=1, class_weight=None, max_queue_size=10, workers=-1, use_multiprocessing=True, shuffle=False)

In [ ]:
history = efficientnetb0_model.fit_generator(train_generator, steps_per_epoch=440, epochs=3, verbose=1, callbacks=None, validation_data=validate_generator, validation_steps=110, validation_freq=1, class_weight=None, max_queue_size=10, workers=-1, use_multiprocessing=True, shuffle=False)